<a href="https://colab.research.google.com/github/Ayushman0Singh/BikeSharingDemandPrediction/blob/main/ModelTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# importing our data handling tools
import numpy as np
import pandas as pd
import numpy as math 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
#mounting drive
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Almabetter/Capstone Projects/Supervised ML-Regression /SeoulBikeData.csv'
df = pd.read_csv(file_path,encoding= 'unicode_escape')

Mounted at /content/drive


In [2]:
#Converting date to date-month-year
import datetime
df['Year'] = pd.DatetimeIndex(df['Date']).year   #getting year column
df['Month'] = pd.DatetimeIndex(df['Date']).month  #getting month column
df['Day'] = pd.DatetimeIndex(df['Date']).day  #getting Day column
df['Day_of_Week'] = pd.DatetimeIndex(df['Date']).weekday #Day of week 
df = df[df['Functioning Day'] == 'Yes']
df.sample(2)

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day,Year,Month,Day,Day_of_Week
1720,10/02/2018,291,16,0.3,17,4.9,2000,-21.9,0.87,0.0,0.0,Winter,No Holiday,Yes,2018,10,2,1
8439,17/11/2018,1018,15,8.9,34,2.9,1787,-6.1,0.52,0.0,0.0,Autumn,No Holiday,Yes,2018,11,17,5


In [31]:
df_pr = df.copy()
df_pr.drop(labels = ['Date','Dew point temperature(°C)', 'Functioning Day' ] ,axis = 1,inplace = True)
df_pr = pd.get_dummies(df_pr, columns=["Seasons", "Holiday", 'Year', 'Month','Day','Day_of_Week', 'Hour' ])

In [46]:
df_pr.sample(2)

,Rented Bike Count,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons_Autumn,Seasons_Spring,...,Hour_14,Hour_15,Hour_16,Hour_17,Hour_18,Hour_19,Hour_20,Hour_21,Hour_22,Hour_23
3631,614,18.0,82,0.7,264,0.11,0.0,0.0,0,1,...,0,0,0,0,0,0,0,0,0,0
2069,37,-0.7,41,0.9,1442,0.00,0.0,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
features = list(df_pr.columns)[1:]
X = df_pr[features]
y = math.sqrt(df_pr['Rented Bike Count'])

In [34]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split( X,y , test_size = 0.2, random_state = 0) 
print(X_train.shape)
print(X_test.shape)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit_transform(X_train)
sc.transform(X_test)

(6772, 89)
(1693, 89)


array([[-1.5488159 , -1.27212732,  0.74002609, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [ 0.85588137,  1.22372383, -0.22378972, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-0.338232  , -0.4891152 ,  2.28213139, ..., -0.21265707,
        -0.20922027, -0.20690275],
       ...,
       [ 0.78999926,  0.19602041, -0.80207921, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-0.66764258,  0.97903254, -0.22378972, ..., -0.21265707,
        -0.20922027, -0.20690275],
       [-1.25234637, -1.37000384,  0.93278925, ..., -0.21265707,
        -0.20922027, -0.20690275]])

In [35]:
#Reguarised linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)
print('r2 for lr =', r2_score((y_test)**2, (y_pred)**2))

r2 for lr = 0.7287162043937694


In [19]:
#lasso/L1
from sklearn.linear_model import Lasso
Lasso = Lasso(alpha = 0.001, max_iter= 1000).fit(X_train,y_train)
y_pred_lasso = Lasso.predict(X_test)
print('r2 for lasso =', r2_score((y_test)**2, (y_pred_lasso)**2))

r2 for lasso = 0.7282643410581838


In [20]:
#Ridge/L2
from sklearn.linear_model import Ridge
Ridge = Ridge(alpha = 0.001, max_iter= 1000).fit(X_train,y_train)
y_pred_ridge = Ridge.predict(X_test)
print('r2 for Ridge =', r2_score((y_test)**2, (y_pred_ridge)**2))

r2 for Ridge = 0.728715293616855


In [40]:
#RMSLE
from sklearn.metrics import mean_squared_log_error
msle=mean_squared_log_error((y_test)**2, (y_pred_lasso)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data Lasso:',rmsle)

RMLSE for the data Lasso: 0.8276824845530378


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [38]:
#Hyperparameter tuning for DT
hyperP_DT = {'max_depth':np.arange(8,100,2),'max_leaf_nodes': np.arange(100,500,10)}
DTregressor1 = GridSearchCV(DecisionTreeRegressor(random_state=88), hyperP_DT, cv=3)
DTregressor1.fit(X_train,y_train) 
y_pred_DTregressor1 = DTregressor1.predict(X_test)
DTregressor1.best_params_

{'max_depth': 20, 'max_leaf_nodes': 170}

In [43]:
#Using the decision tree of the best parameters. 
print('r2 for DT1 =', r2_score((y_test)**2, (y_pred_DTregressor1)**2))
#M

#RMSLE
msle=mean_squared_log_error((y_test)**2, (y_pred_DTregressor1)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data DT1:',rmsle)
print('RMLSE for the data DT1 :',rmsle)
#Tree Properties
# print('Tree Depth',DTregressor1.get_depth())
# print('Tree Max Nodes', DTregressor1.get_n_leaves())

r2 for DT1 = 0.7594398739805168
RMLSE for the data DT1: 0.6058124788305737
RMLSE for the data DT1 : 0.6058124788305737


In [44]:
#Decision Tree Regressor
DTregressor = DecisionTreeRegressor(max_leaf_nodes = 400, random_state=88)
DTregressor.fit(X_train,y_train)
y_pred_DTregressor = DTregressor.predict(X_test)

In [45]:
print('r2 for DT =', r2_score((y_test)**2, (y_pred_DTregressor)**2))
print('r2 on train set DT =', DTregressor.score(X_train,y_train) )
#M

#RMSLE
msle=mean_squared_log_error((y_test)**2, (y_pred_DTregressor)**2)
rmsle=np.sqrt(msle)
print('RMLSE for the data DT:',rmsle)
print('RMLSE for the data DT :',rmsle)
#Tree Properties
print('Tree Depth',DTregressor.get_depth())
print('Tree Max Nodes', DTregressor.get_n_leaves())

r2 for DT = 0.7569342826519919
r2 on train set DT = 0.9431212721545797
RMLSE for the data DT: 0.6108428588893801
RMLSE for the data DT : 0.6108428588893801
Tree Depth 26
Tree Max Nodes 400


In [30]:
#RF
RFclassifier = RandomForestRegressor()
grid_values = {'n_estimators':[50, 80,  100], 'max_depth':[3, 5, 7]}
classifier = GridSearchCV(RFclassifier, param_grid = grid_values, scoring = 'neg_mean_squared_log_error', cv=5)

# Fit the object to train dataset
classifier.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
45 fits failed out of a total of 45.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
45 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 367, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py", line 734, in _validate_y_class_weight
    check_classification_targets(y)
  File "/

ValueError: ignored